<br>

<h1 style="text-align:center;">Hyperparameter Search</h1>

<br>

## Initial Setup

---

In [1]:
import os
import itertools
import json
import wandb
from typing import Dict, Any
import copy
import signal
from train_ot2_rl import main as train_main

In [2]:
import wandb
wandb.tensorboard.patch(root_logdir="./runs")

<br>

## Hyperparameters to Search

---

In [3]:
# Base hyperparameters matching the training script
BASE_HYPERPARAMS = {
    # Learning parameters
    "learning_rate": 1e-4,
    "buffer_size": 100000,
    "batch_size": 256,
    "tau": 0.005,
    "gamma": 0.995,
    "train_freq": (1, "step"),
    "n_envs": 1,
    "gradient_steps": 1,
    
    # Architecture
    "actor_layers": [512, 512],
    "critic_layers": [512, 512],
    "activation_fn": "relu",
    "net_arch": "default",
    
    # Training
    "total_timesteps": 10_000,  # Reduced for search
    "learning_starts": 100,
    "ent_coef": "auto_1.0",
    
    # Environment
    "max_episode_steps": 500,
    "normalize_observations": True,
    "normalize_rewards": True,
    "reward_scale": 1.0,
    "clip_obs": 5.0,
    "clip_rewards": 5.0,
    
    # Evaluation
    "eval_freq": 10_000,  # Make sure we get at least one evaluation
    "n_eval_episodes": 5,
    "eval_deterministic": True
}

In [4]:
# Define parameter grid for search
PARAM_GRID = {
    "learning_rate": [3e-5, 1e-4, 3e-4],
    "max_episode_steps": [500, 1000, 2000],
    "buffer_size": [100000, 300000, 500000],
}

In [5]:
def generate_combinations() -> list:
    """Generate all combinations of hyperparameters to test."""
    keys = PARAM_GRID.keys()
    values = PARAM_GRID.values()
    combinations = list(itertools.product(*values))
    
    # Convert to list of dictionaries
    param_combinations = []
    for combo in combinations:
        params = dict(zip(keys, combo))
        # Create full config by updating base params
        full_config = copy.deepcopy(BASE_HYPERPARAMS)
        full_config.update(params)
        param_combinations.append(full_config)
    
    return param_combinations

In [6]:
# Define the hyperparameter search space
def create_hyperparameter_grid():
    """Create a simplified grid of hyperparameters to search"""
    return [
        {
            "learning_rate": lr,
            "buffer_size": bs,
            "max_episode_steps": 500  # Keep this constant
        }
        for lr in [3e-5, 1e-4, 3e-4]
        for bs in [100000, 300000]
    ]

<br>

## Run Trials

---



In [7]:
def run_trial(hyperparams, trial_num, param_name):
    """Modified trial runner to properly capture wandb metrics"""
    trial_dir = f'./trials/trial_{trial_num}'
    os.makedirs(trial_dir, exist_ok=True)
    
    try:
        # Initialize wandb with project name based on hyperparameter
        wandb.init(
            project=f"ot2_rl_hyperparameters_{param_name}",
            name=f"{param_name}_{hyperparams[param_name]}",
            config=hyperparams,
            sync_tensorboard=True,
            save_code=True,
        )
        
        # Run training and get the final evaluation reward
        final_eval_reward = train_main(custom_hyperparams=hyperparams, trial_dir=trial_dir)
        
        # Get the wandb run history
        if wandb.run is not None:
            history = wandb.run.history.rows
            if history:  # Get the last row of metrics
                last_metrics = history[-1]
                trial_results = {
                    'trial_num': trial_num,
                    'status': 'completed',
                    'final_distance_mm': last_metrics.get('distance/current', None),
                    'steps_taken': last_metrics.get('training/timesteps', None),
                    'min_distance': last_metrics.get('current/min_distance', None),
                    'episode_reward': last_metrics.get('current/episode_reward', None),
                    'distance_improvement': last_metrics.get('distance/improvement', None),
                    'steps_per_second': last_metrics.get('training/steps_per_second', None),
                }
            else:
                trial_results = {
                    'trial_num': trial_num,
                    'status': 'completed',
                    'final_distance_mm': final_eval_reward,
                    'steps_taken': hyperparams['total_timesteps']
                }
                
            # Save trial results to JSON
            with open(f'{trial_dir}/results.json', 'w') as f:
                json.dump(trial_results, f)
                
    except Exception as e:
        print(f"Error in trial {trial_num}: {str(e)}")
        # Record failed trial
        trial_results = {
            'trial_num': trial_num,
            'status': 'failed',
            'error': str(e)
        }
        with open(f'{trial_dir}/results.json', 'w') as f:
            json.dump(trial_results, f)
    
    finally:
        if wandb.run is not None:
            wandb.finish()

In [1]:
def main():
    """Run hyperparameter search."""
    # Generate parameter combinations
    param_combinations = generate_combinations()
    
    # Create trials directory
    os.makedirs('./trials', exist_ok=True)
    
    # Run trials for each hyperparameter separately
    for param_name in PARAM_GRID.keys():
        print(f"\nRunning trials for {param_name}")
        
        # Filter combinations to vary only this parameter
        base_params = copy.deepcopy(BASE_HYPERPARAMS)
        trial_num = 1
        
        for value in PARAM_GRID[param_name]:
            params = copy.deepcopy(base_params)
            params[param_name] = value
            
            print(f"\nStarting trial {trial_num} for {param_name}={value}")
            
            # Clear any existing progress bars before starting new trial
            try:
                from tqdm import tqdm
                for inst in tqdm._instances:
                    inst.close()
                tqdm._instances.clear()
            except:
                pass
                
            run_trial(params, trial_num, param_name)
            trial_num += 1

if __name__ == "__main__":
    main()

pybullet build time: Nov 28 2023 23:52:03
2025-01-21 22:23:55.833071: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-21 22:23:55.848363: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737494635.864567  155608 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737494635.869826  155608 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 22:23:55.888287: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow bina


Running trials for learning_rate

Starting trial 1 for learning_rate=3e-05


wandb: Currently logged in as: soheil-mpg. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


2025-01-21 22:24:00.505183: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-21 22:24:00.514387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737494640.525949  155715 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737494640.529586  155715 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 22:24:00.541574: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Normalization stats loaded successfully
Previous best model loaded successfully
Logging to runs/1o282s52/SAC_85


...


╔══════════════════════════════════════════════════╗
║                Episode 18 Summary                ║
╠══════════════════════════════════════════════════╣
║Initial Position:          0.073,   0.090,   0.119 ║
║Current Position:         -0.128,   0.204,   0.230 ║
║Goal Position:            -0.117,   0.205,   0.220 ║
║Initial Distance (m):                       0.2441 ║
║Final Distance (m):                         0.0146 ║
║Distance Improvement (m):                   0.2295 ║
║Steps Taken:                                   501 ║
║Total Reward:                               387.96 ║
║Average Reward/Step:                          0.77 ║
║Status:                        Max Steps Reached ⌛ ║
╚══════════════════════════════════════════════════╝

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 502      |
|    ep_rew_mean     | 395      |
| time/              |          |
|    episodes        | 17       |
|    fps             | 41       |
|    tim


╔══════════════════════════════════════════════════╗
║                Episode 19 Summary                ║
╠══════════════════════════════════════════════════╣
║Initial Position:          0.073,   0.090,   0.119 ║
║Current Position:          0.112,  -0.108,   0.236 ║
║Goal Position:             0.120,  -0.103,   0.225 ║
║Initial Distance (m):                       0.2255 ║
║Final Distance (m):                         0.0138 ║
║Distance Improvement (m):                   0.2117 ║
║Steps Taken:                                   501 ║
║Total Reward:                               390.34 ║
║Average Reward/Step:                          0.78 ║
║Status:                        Max Steps Reached ⌛ ║
╚══════════════════════════════════════════════════╝

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 502      |
|    ep_rew_mean     | 394      |
| time/              |          |
|    episodes        | 18       |
|    fps             | 41       |
|    tim


╔══════════════════════════════════════════════════╗
║                Episode 20 Summary                ║
╠══════════════════════════════════════════════════╣
║Initial Position:          0.073,   0.090,   0.119 ║
║Current Position:          0.016,  -0.004,   0.270 ║
║Goal Position:             0.023,   0.000,   0.258 ║
║Initial Distance (m):                       0.1732 ║
║Final Distance (m):                         0.0140 ║
║Distance Improvement (m):                   0.1592 ║
║Steps Taken:                                   501 ║
║Total Reward:                               400.23 ║
║Average Reward/Step:                          0.80 ║
║Status:                        Max Steps Reached ⌛ ║
╚══════════════════════════════════════════════════╝

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 502       |
|    ep_rew_mean     | 395       |
| time/              |           |
|    episodes        | 19        |
|    fps             | 41        |
|

/home/bluestargaze/miniconda3/envs/cv_rl_pipeline/lib/python3.9/site-packages/stable_baselines3/common/save_util.py
:278: UserWarning: Path 'checkpoints/sac_model_replay_buffer_10000_steps.pkl' exists, will overwrite it.
  warnings.warn(f"Path '{path}' exists, will overwrite it.")

╔══════════════════════════════════════════════════╗